In [8]:
PI = 3.141592

In [9]:
def abs(x):
    return -x if x < 0 else x

def sqrt(y, tol=1e-12, max_iter=100):
    if y < 0:
        raise ValueError("sqrt: argumento negativo")
    if y == 0:
        return 0.0
    x = y if y >= 1 else 1.0  
    for _ in range(max_iter):
        xn = 0.5 * (x + y / x)
        if abs(xn - x) <= tol:
            return xn
        x = xn
    return x 


In [10]:
def _wrap_to_pi(x):
    # Reduce a [-pi, pi] sin usar modulo
    two_pi = 2.0 * PI
    # aproximación de modulo por restas controladas
    k = int(x / two_pi)
    x = x - k * two_pi
    if x > PI:
        x -= two_pi
    if x < -PI:
        x += two_pi
    return x

In [11]:
def sin(x):
    x = _wrap_to_pi(x)
    term = x
    s = term
    x2 = x * x
    n = 1
    while True:
        n += 1  
        term *= -x2 / ((2*n-2) * (2*n-3))
        s_new = s + term
        if abs(term) < 1e-14:
            return s_new
        s = s_new
        if n > 35:
            return s

def cos(x):
    x = _wrap_to_pi(x)
    term = 1.0
    s = term
    x2 = x * x
    n = 1
    while True:
        term *= -x2 / ((2*n-1) * (2*n))
        s_new = s + term
        if abs(term) < 1e-14:
            return s_new
        s = s_new
        n += 1
        if n > 35:
            return s

In [12]:
def exp(x):
    if x < 0:
        return 1.0 / exp(-x)
    term = 1.0
    s = 1.0
    n = 1
    while True:
        term *= x / n
        s_new = s + term
        if abs(term) < 1e-14:
            return s_new
        s = s_new
        n += 1
        if n > 60:
            return s

In [13]:
def G(x, const_c):
    x1, x2, x3 = x
    # g1
    g1 = (cos(x2 * x3) + 0.5) / 3.0
    # g2 (rama +)
    rad = x1 * x1 + sin(x3) + 1.06
    if rad < 0:
        rad = 0.0
    g2 = -0.1 + (1.0 / 9.0) * sqrt(rad)
    # g3
    g3 = -(exp(-(x1 * x2)) + const_c) / 20.0
    def clip(y): 
        return -1.0 if y < -1.0 else (1.0 if y > 1.0 else y)
    return (clip(g1), clip(g2), clip(g3))

In [14]:
def solve(x0=(0.0, 0.0, 0.0), tol=1e-8, max_iter=100000):
    const_c = (10.0 * PI - 3.0) / 3.0  # (10π - 3)/3
    x = tuple(float(v) for v in x0)
    for it in range(1, max_iter + 1):
        xn = G(x, const_c)
        # criterio de paro: norma infinito
        err = max(abs(xn[0]-x[0]), abs(xn[1]-x[1]), abs(xn[2]-x[2]))
        if err <= tol:
            return {"x": xn, "iterations": it, "error": err, "converged": True}
        x = xn
    return {"x": x, "iterations": max_iter, "error": err, "converged": False}

In [15]:
result = solve(x0=(0.0, 0.0, 0.0), tol=1e-8, max_iter=200000)
print(result)

{'x': (0.49999963262764835, 0.0028358795791037783, -0.5235278197493726), 'iterations': 6, 'error': 8.8091275385338e-09, 'converged': True}
